In [ ]:
Threads.nthreads() = 10

In [ ]:
include("ftdt_mirror.jl")
using .ftdt
using ProgressBars

########################################################################
# Se define la geometría en la ventana numérica principal
#######################################################################
und = 1*10^-6;                       # Unidad espacial
unt = 1*10^0;                       # Unidad temporal

epsz = (8.854187817*10^-12)*((und^3)/(unt^4)); 
muz = (4*pi*1e-7)*((unt^2)/und);
etaz=sqrt(muz/epsz);
cc =  sqrt(1/(epsz*muz));

freq=3.3e+14*(unt);                        # Frecuencia de la fuente de campo
lambda=cc/freq;                     # Longitud de onda de la fuente de campo 
omega=2.0*pi*freq;                  # Frecuencia en radiandes

dx = lambda/20;#(100e-3)*und;                             # Diferencial espacial

ventNum = round.([30*10^-3 15*10^-3]/und*dx,RoundUp)

ie= Int(ventNum[2]);                                     # nodos en y
je= Int(ventNum[1]);  

"""
COSOS
"""


CFL = 0.99                          # Constante de Courant
dt = CFL / (cc * sqrt((1/dx^2) + (1/dx^2)))  # Diferencial temporal
tau = Int(round(1 / freq / dt))     # Nodos por periodo
nmax = tau * 60                     # Cantidad total de nodos temporales
ie = Int(round(15e-3 / und * dx))   # nodos en y
je = Int(round(30e-3 / und * dx))   # nodos en x
nx = je
ny = ie
nt = nmax

parametros = [nx,ny,nt,dx,dt,und,unt]

# Se agregan objetos en el medio
casoMaterial = "vacio";
if casoMaterial=="vacio"
  idx = 1:je;
  idy = 1:ie;
  medioProp = epsz;
end


In [ ]:

# Lente de Luneburg
f = 1;
# En nodos computacionales
centr = [div(ie,2),div(je,2)];
rad = Int(round(( 1*lambda )/dx,RoundUp));

idx = centr[2]-rad:centr[2]+rad;
idy = centr[1]-rad:centr[1]+rad;

domx = dx*collect(idx);
domy = dx*collect(idy);
aux = length(domx);

domR = sqrt.( ((ones(aux,1).*domx').-centr[2]*dx).^2 + ((ones(1,aux).*domy).-centr[1]*dx).^2 );
n = real.((1/f)* sqrt.(Complex.( (1).+(f^2).-((domR./rad*dx).^2) )));
n[domR.>rad*dx].=1;
medioProp = epsz*(n.^2);
    


In [ ]:
# Initialize mirrorMatrix to match the size of hz
mirrorMatrix = zeros(Int, 682, 1363)  # Adjusted size to match hz

# Recalculate the center and size to fit within the new dimensions
centerX = Int(round(size(mirrorMatrix, 2) / 2))
centerY = Int(round(size(mirrorMatrix, 1) / 2))
mirrorSize = Int(round(min(size(mirrorMatrix, 1), size(mirrorMatrix, 2)) / 10))

# Define start and end indices for the mirror
startX = centerX - Int(mirrorSize / 2)
endX = centerX + Int(mirrorSize / 2)
startY = centerY - Int(mirrorSize / 2)
endY = centerY + Int(mirrorSize / 2)

# Ensure indices are within bounds
startX = max(startX, 1)
startY = max(startY, 1)
endX = min(endX, size(mirrorMatrix, 2))
endY = min(endY, size(mirrorMatrix, 1))

# Set mirror region to 1
mirrorMatrix[startY:endY, startX:endX] .= 1

# Confirm the size and position
@show size(mirrorMatrix)
@show startX, endX, startY, endY



In [ ]:
# Columan de lentes de Luneburg
nL = 5;
rad = 60; #div(ie-101,nL);
f = 1;
#f = 1.5*rad;

auxCr = (2*rad.*[collect(-div(nL-1,2):1:div(nL-1,2)) zeros(nL,1)])
ctrA = [div(ie,2) div(je,4)];
centr = ones(nL,1).*ctrA;
centr[:,1] = centr[:,1].+auxCr[:,1];
centr[:,2] = centr[:,2].+auxCr[:,2];
centr = Int.(centr);

n = zeros(ie,je);
for itl = 1:nL
  idx = centr[itl,2]-rad-1:centr[itl,2]+rad-1;
  idy = centr[itl,1]-rad-1:centr[itl,1]+rad-1;
  
  domx = dx*collect(idx);
  domy = dx*collect(idy);
  aux = length(domx);
  
  domR = sqrt.( ((ones(aux,1).*domx').-centr[itl,2]*dx).^2 + ((ones(1,aux).*domy).-centr[itl,1]*dx).^2 );
  nAux = real.((1/f)* sqrt.(Complex.( (1).+(f^2).-((domR./rad*dx).^2) )));
  #alf = 0.74;
  #nAux = real.((1/f)* sqrt.(Complex.( (rad^2).+(f^2).-alf*(domR.^2) )));
  n[idy,idx] = n[idy,idx].+nAux.*(domR.<rad*dx);
end
n[n.==0].=1;
idx = 1:je;
idy = 1:ie;
medioProp = epsz*(n.^2);

# save(File(format"JLD",string(pwd(),"\\gifMagico\\casoMaterialLuneburg.jld")),"caso",casoMaterial,"f",f,"rad",rad,"centr",centr)


In [ ]:
# Display mirrorMatrix dimensions and placement information for confirmation
@show size(mirrorMatrix)
@show startX, endX, startY, endY
@show size(medioProp)

matrices = simular(medioProp, mirrorMatrix, idx, idy, nmax)

In [ ]:
using Plots, Colors
using FileIO
#using CairoMakie, Colors            # Graficas
#CairoMakie.activate!(type = "svg")
direc = "/home/jay/Photonics/FTDTgifMagico"

# Vista previa de objeto
fsx = (1/parametros[4])*parametros[6];
fsy = (1/parametros[4])*parametros[6];
domx = fsx*collect(1:parametros[1]);
domy = fsy.*collect(1:parametros[2]);

# Debes asegurarte que parametros[1] y parametros[2] son enteros
nx = Int(round(parametros[1]))
ny = Int(round(parametros[2]))

# Ahora puedes usar nx y ny para definir tus matrices
initial_info = matrices[Int(parametros[3])]

ny, nx = size(initial_info)
inte = zeros(ny, nx)

for itint in ProgressBar(1:1730)
        index = Int(parametros[3]-itint)
        info = matrices[index]
        info = info./maximum(info);
        inte = inte .+ abs.(info).^2;
end

plotly()  # Using Plotly for interactive plots, or choose another backend like gr()

# Initialize the plotting
grafZoom2 = plot(layout=grid(1,1), colorbar=:top)

# Adding heatmap to the plot
heatmap!(grafZoom2[1], domx, domy, (1/1740) .* inte, colorbar=:top, xlabel="x [mts]", ylabel="y [mts]",
         title="Intensidad", aspect_ratio=:equal,
         # Uncomment and modify the following lines as needed
         # xlims=(limX[1], limX[2]), ylims=(limY[1], limY[2]),
         c = :berlin)

# Uncomment if you want to add additional plot features, such as overlaying lines
# plot!(grafZoom2[1], fsx*materiales[1]*cos.(th) .+ centr[2]*fsx, materiales[1]*fsx*sin.(th) .+ centr[1]*fsx,
#       label=false, linecolor=:black, linewidth=1)

# Display the plot in the notebook
display(grafZoom2)


In [ ]:
# Skectch final
ref = 1;
itn = Int(parametros[3]) - 500;#parametros[3]; 
info = matrices[itn]
info = info./maximum(info);
deltaY = domy[2] - domy[1]  # Assuming uniform spacing, find the step
domy = [domy; domy[end] + deltaY]  # Append a new value that continues the interval

grafFinal = heatmap(domx,domy,info,clims=(-ref,ref),xlabel="x [mts]",ylabel="y [mts]",
                    aspect_ratio=:equal,
                    xlims=(0,fsx*parametros[1]),ylims=(0,fsy*parametros[2]),
                    c = :berlin
                    )
# plot!(grafFinal,fsx*materiales[1]*cos.(th).+centr[2]*fsx, materiales[1]*fsx*sin.(th).+centr[1]*fsx,label=false,
#                     linecolor=:white ,linewidth=0.5
#                     )
annotate!(grafFinal,[(0.026,0.014,(string("Iteración: ",itn),10,:white))])
annotate!(grafFinal,[(0.026,0.001,(string("t= ",round(itn*parametros[5]*parametros[7],digits=16)," s"),10,:white))])

display(grafFinal)


In [ ]:
gr()  # Switch to GR backend for better handling of animations

# Function to create and display an animation in the notebook
function display_animation(matrices, domx, domy, fsx, fsy, parametros)

    # IT IS SLOOOOOW
    key_subset = sort(collect(keys(matrices)))[1:50:end]
    
    
    anim = @animate for itn in ProgressBar(key_subset)
        info = matrices[itn]
        info = info ./ maximum(info)

        # println(size(info))
        # println(length(domx), " ", length(domy))

        # Cut last n domy values to match the size of info

        domy_extended = domy

        grafFinal = heatmap(domx, domy_extended, info, clims=(-1, 1), xlabel="x [mts]", ylabel="y [mts]",
                            aspect_ratio=:equal,
                            xlims=(0, fsx*parametros[1]), ylims=(0, fsy*parametros[2]),
                            c = :berlin)

        # Optional plot features
        # plot!(grafFinal, fsx*materiales[1]*cos.(th).+centr[2]*fsx, materiales[1]*fsx*sin.(th).+centr[1]*fsx,
        #               label=false, linecolor=:white, linewidth=0.5)

        annotate!(grafFinal, [(0.026, 0.014, ("Iteración: $itn", 10, :white))])
        annotate!(grafFinal, [(0.026, 0.001, ("t= $(round(itn*parametros[5]*parametros[7], digits=16)) s", 10, :white))])
    end
    gif(anim, "animation.gif", fps = 120)  # Adjust fps according to your preference
end

# Call the function to create and display the animation in the notebook
animation = display_animation(matrices, domx, domy, fsx, fsy, parametros)


In [ ]:


# Zoom en la frontera de los lentes
ref = 1;
itn = parametros[3];#parametros[3];
info = load( string(direc,"/matHz-",itn,".jld"), "hz");
info = info./maximum(info);
limY = fsx*round.( [centr[1]-1.5materiales[1],centr[1]+1.5materiales[1]],RoundUp);
limX = fsx*round.( [centr[2]-1.5materiales[1],centr[2]+1.5materiales[1]],RoundUp);

grafZoom1 = plot(layout=grid(1,1),colorbar=:top,clims=(-ref,ref))
heatmap!(grafZoom1[1],domx,domy,info,colorbar=:top,xlabel="x [mts]",ylabel="y [mts]",
        title="Campo Hz",aspect_ratio=:equal,
        xlims=(limX[1],limX[2]),ylims=(limY[1],limY[2]),
        c = :berlin
        );
plot!(grafZoom1[1],fsx*materiales[1]*cos.(th).+centr[2]*fsx, materiales[1]*fsx*sin.(th).+centr[1]*fsx,label=false,
        linecolor=:white ,linewidth=1
        );
savefig(grafZoom1,string("campoHz",numCarp,".pdf"))

# Cortes Transversales
ref = 1;
itn = parametros[3];#parametros[3];
refT = round(itn*parametros[5]*parametros[7],digits=16);
info = load( string(direc,"\\matHz-",itn,".jld"), "hz");
info = info./maximum(info);
inte = abs.(info).^2;

grafCT = plot(layout=grid(2,1))
plot!(grafCT[1],domx,info[div(parametros[2],2),:],aspect_ratio=:auto,xlims=(0,parametros[1]*fsx),ylims=(-1,1),
        title="Amplitud Campo Hz",xlabel="x [mts]",ylabel="Amplitud",label = string("t=",refT," s"),
        linecolor=:purple)
plot!(grafCT[2],domx,inte[div(parametros[2],2),:],aspect_ratio=:auto,xlims=(0,parametros[1]*fsx),ylims=(0,1),
        title="Intensidad Campo Hz",xlabel="x [mts]",ylabel="Intensidad",label = string("t=",refT," s"),
        linecolor=:purple)

savefig(grafCT,string("campoHz",numCarp,"_CT.pdf"))

# Creación del gif
ref = 1;
@time begin
anim = @animate for itn = 1:4:parametros[3]
    info = load( string(direc,"\\matHz-",itn,".jld"), "hz");
    info = info./maximum(info);
    grafFinal = heatmap(domx,domy,info,clims=(-ref,ref),xlabel="x [mts]",ylabel="y [mts]",
                        aspect_ratio=:equal,
                        xlims=(0,fsx*parametros[1]),ylims=(0,fsy*parametros[2]),
                        c = :berlin
                        )
    plot!(grafFinal,fsx*materiales[1]*cos.(th).+centr[2]*fsx, materiales[1]*fsx*sin.(th).+centr[1]*fsx,label=false,
                        linecolor=:white ,linewidth=0.5
                        )
    annotate!(grafFinal,[(0.026,0.014,(string("Iteración: ",itn),10,:white))])
    annotate!(grafFinal,[(0.026,0.001,(string("t= ",round(itn*parametros[5]*parametros[7],digits=16)," s"),10,:white))])
end
end

gif(anim,string("anim",numCarp,".gif"), fps = 120)